In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .config("spark.executor.memory", "4g") \
    	.config("spark.driver.memory", "4g") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 21:05:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
df_green.createOrReplaceTempView ('green_trips')

In [5]:
df_green_revenue = \
    spark.sql(""" 
                select
                    date_trunc('hour', lpep_pickup_datetime)  as hour,
                    PULocationID as zones,
                    sum(total_amount) as revenue_monthly_total_amount,
                    count(*) as number_records
                from green_trips
                where lpep_pickup_datetime >= '2020-01-01 00:00:00'
                and lpep_pickup_datetime is not null
                group by 1, 2
          
                ;
                """)

In [6]:
df_green_revenue.show()

+-------------------+-----+----------------------------+--------------+
|               hour|zones|revenue_monthly_total_amount|number_records|
+-------------------+-----+----------------------------+--------------+
|2020-01-28 19:00:00|  134|          193.61000000000007|            17|
|2020-01-22 19:00:00|   65|           657.0300000000001|            41|
|2020-01-27 08:00:00|   17|                       85.56|             4|
|2020-01-02 09:00:00|   66|          229.39999999999998|            12|
|2020-01-02 12:00:00|   89|          310.28000000000003|            14|
|2020-01-07 12:00:00|   66|                       179.5|             9|
|2020-01-03 08:00:00|  223|          165.90000000000003|             9|
|2020-01-17 10:00:00|   41|           638.2699999999999|            49|
|2020-01-02 11:00:00|   26|                       198.6|             7|
|2020-01-12 20:00:00|  247|          36.900000000000006|             3|
|2020-01-12 15:00:00|   10|                       75.92|        

In [7]:
df_green_revenue.repartition(4).write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [9]:
df_yellow.createOrReplaceTempView ('yellow_trips')

In [10]:
df_yellow_revenue = \
    spark.sql(""" 
                select
                    date_trunc('hour', tpep_pickup_datetime)  as hour,
                    PULocationID as zones,
                    sum(total_amount) as revenue_monthly_total_amount,
                    count(*) as number_records
                from yellow_trips
                where tpep_pickup_datetime >= '2020-01-01 00:00:00'
                and tpep_pickup_datetime is not null
                group by 1, 2
          
                ;
                """)

In [11]:
df_yellow_revenue.show()

+-------------------+-----+----------------------------+--------------+
|               hour|zones|revenue_monthly_total_amount|number_records|
+-------------------+-----+----------------------------+--------------+
|2020-01-10 06:00:00|  144|                      149.51|             9|
|2020-01-29 03:00:00|   13|                      248.89|             6|
|2020-01-13 21:00:00|  255|                      108.98|             8|
|2020-01-04 12:00:00|  249|           2245.569999999999|           152|
|2020-01-02 15:00:00|  236|           7382.910000000018|           504|
|2020-01-31 21:00:00|   50|          1784.4399999999994|           104|
|2020-01-07 12:00:00|   68|           3046.229999999997|           181|
|2020-01-02 09:00:00|  161|          4299.9299999999985|           242|
|2020-01-23 17:00:00|  233|           4642.769999999999|           235|
|2020-01-06 22:00:00|  158|           923.2100000000003|            57|
|2020-01-15 07:00:00|   13|          2432.6299999999997|        

In [12]:
df_yellow_revenue.repartition(4).write.parquet('data/report/revenue/yellow', mode='overwrite')

In [13]:
df_green_revenue_tmp = df_green_revenue \
                        .withColumnRenamed('revenue_monthly_total_amount', 'green_amount') \
                        .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
                        .withColumnRenamed('revenue_monthly_total_amount', 'yellow_amount') \
                        .withColumnRenamed('number_records', 'yellow_number_records')

In [14]:
combined_df = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zones'], how='outer')

In [15]:
combined_df.show()

+-------------------+-----+------------------+--------------------+------------------+---------------------+
|               hour|zones|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+-----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|    3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|    4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|    7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|   12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|   37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|   40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:0

In [16]:
combined_df.write.parquet('data/report/revenue/total', mode='overwrite')

In [17]:
df_zones = spark.read.parquet('zones/')

In [19]:
df_result = combined_df.join(df_zones, combined_df.zones == df_zones.LocationID)

In [20]:
df_result.drop('LocationID', 'zones').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|                Zone|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|              null|                null|              25.0|                    1|    Bronx|Allerton/Pelham G...|   Boro Zone|
|2020-01-01 00:00:00|              null|                null|1004.3000000000002|                   57|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00| 769.7299999999996|                  45| 455.1700000000001|                   38|   Queens|             Astoria|   Boro Zone|
|2020-01-01 00:00:00|              null|                null|             107.0|                    6|Manhattan|      

In [21]:
df_result.drop('LocationID').write.parquet('tmp/revenue_zones')